# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Data & Feature views</span>


<span style="font-width:bold; font-size: 1.4rem;">This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups
2. Create Feature Views
3. Create Training Dataset with training and test splits
 

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5318


Connected. Call `.close()` to terminate connection gracefully.


In [2]:
streamflow_fg = fs.get_or_create_feature_group(
    name = 'streamflow_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

In [3]:
query = streamflow_fg.select_all().join(weather_fg.select_all())

In [4]:
query.read()

2022-12-23 12:35:46,898 INFO: USE `irisflowers_featurestore`
2022-12-23 12:35:47,511 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`streamflow` `streamflow`, `fg1`.`place` `place`, `fg1`.`date` `join_pk_date`, `fg1`.`place` `join_pk_place`, `fg1`.`date` `join_evt_date`, `fg0`.`temperature_2m_max` `temperature_2m_max`, `fg0`.`temperature_2m_min` `temperature_2m_min`, `fg0`.`precipitation_sum` `precipitation_sum`, `fg0`.`rain_sum` `rain_sum`, `fg0`.`snowfall_sum` `snowfall_sum`, `fg0`.`precipitation_hours` `precipitation_hours`, `fg0`.`windspeed_10m_max` `windspeed_10m_max`, `fg0`.`windgusts_10m_max` `windgusts_10m_max`, `fg0`.`winddirection_10m_dominant` `winddirection_10m_dominant`, `fg0`.`et0_fao_evapotranspiration` `et0_fao_evapotranspiration`, RANK() OVER (PARTITION BY `fg0`.`date`, `fg0`.`place`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `irisflowers_featurestore`.`streamflow_fg_1` `fg1`
INNER JOIN `irisflowers_featurestore`.`weath

,date,streamflow,place,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration
0,1671663600000,30.4489,Abisko,-6.4,-11.2,0.6,0.0,0.42,6.0,16.6,36.0,224.0,0.04
1,1671663600000,5.8187,Spånga,5.2,3.6,3.0,1.7,0.00,12.0,18.0,37.8,215.0,0.08
2,1671663600000,5.0213,Uppsala,4.3,3.1,1.0,0.2,0.00,5.0,18.0,32.0,208.0,0.06
3,1671231600000,33.2854,Abisko,-7.2,-9.9,0.3,0.0,0.21,3.0,6.9,26.6,293.0,0.00
4,1671231600000,5.3238,Spånga,0.7,-7.9,1.7,0.1,1.12,8.0,21.3,48.6,257.0,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,1609542000000,15.5454,Spånga,1.0,0.2,0.0,0.0,0.00,0.0,8.5,18.0,27.0,0.08
2064,1609542000000,20.4436,Uppsala,1.1,0.2,0.6,0.2,0.42,4.0,10.9,22.7,49.0,0.04
2065,1609455600000,28.7626,Abisko,-3.3,-10.9,0.0,0.0,0.00,0.0,11.3,38.9,123.0,0.00
2066,1609455600000,16.2807,Spånga,1.9,-0.3,0.8,0.3,0.35,7.0,11.5,24.5,122.0,0.08


--- 

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

In [5]:
# category_cols = ['city','date','conditions','aqi']

# mapping_transformers = {col_name:fs.get_transformation_function(name='standard_scaler') for col_name in col_names if col_name not in category_cols}
# category_cols = {col_name:fs.get_transformation_function(name='label_encoder') for col_name in category_cols if col_name not in ['date','aqi']}

# mapping_transformers.update(category_cols)

Create the `Feature View`:

In [7]:
feature_view = fs.create_feature_view(
    name = 'streamflow_fv',
    version = 1,
    #transformation_functions = mapping_transformers,
    query = query
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5318/fs/5238/fv/streamflow_fv/version/1


For now `Feature View` is saved in Hopsworks and you can retrieve it using `FeatureStore.get_feature_view()`.

In [8]:
feature_view = fs.get_feature_view(
    name = 'streamflow_fv',
    version = 1
)

In [12]:
from datetime import datetime, timedelta

def get_yesterday(the_date, days_to_subtract=1):
    the_date = (the_date - timedelta(days=days_to_subtract)).strftime("%Y-%m-%d")
    the_date = datetime.strptime(the_date, '%Y-%m-%d')
    return the_date

start_date = get_yesterday(datetime.today(), 1)
X = feature_view.get_batch_data(start_time=start_date)

2022-12-23 12:56:33,480 INFO: USE `irisflowers_featurestore`
2022-12-23 12:56:34,067 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`streamflow` `streamflow`, `fg1`.`place` `place`, `fg1`.`date` `join_pk_date`, `fg1`.`place` `join_pk_place`, `fg1`.`date` `join_evt_date`, `fg0`.`temperature_2m_max` `temperature_2m_max`, `fg0`.`temperature_2m_min` `temperature_2m_min`, `fg0`.`precipitation_sum` `precipitation_sum`, `fg0`.`rain_sum` `rain_sum`, `fg0`.`snowfall_sum` `snowfall_sum`, `fg0`.`precipitation_hours` `precipitation_hours`, `fg0`.`windspeed_10m_max` `windspeed_10m_max`, `fg0`.`windgusts_10m_max` `windgusts_10m_max`, `fg0`.`winddirection_10m_dominant` `winddirection_10m_dominant`, `fg0`.`et0_fao_evapotranspiration` `et0_fao_evapotranspiration`, RANK() OVER (PARTITION BY `fg1`.`date`, `fg1`.`place`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `irisflowers_featurestore`.`streamflow_fg_1` `fg1`
INNER JOIN `irisflowers_featurestore`.`weath

In [15]:
start_date = get_yesterday(datetime.today(), 2)
X = feature_view.get_batch_data(start_time=start_date)
X

2022-12-23 12:57:44,828 INFO: USE `irisflowers_featurestore`
2022-12-23 12:57:45,366 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`streamflow` `streamflow`, `fg1`.`place` `place`, `fg1`.`date` `join_pk_date`, `fg1`.`place` `join_pk_place`, `fg1`.`date` `join_evt_date`, `fg0`.`temperature_2m_max` `temperature_2m_max`, `fg0`.`temperature_2m_min` `temperature_2m_min`, `fg0`.`precipitation_sum` `precipitation_sum`, `fg0`.`rain_sum` `rain_sum`, `fg0`.`snowfall_sum` `snowfall_sum`, `fg0`.`precipitation_hours` `precipitation_hours`, `fg0`.`windspeed_10m_max` `windspeed_10m_max`, `fg0`.`windgusts_10m_max` `windgusts_10m_max`, `fg0`.`winddirection_10m_dominant` `winddirection_10m_dominant`, `fg0`.`et0_fao_evapotranspiration` `et0_fao_evapotranspiration`, RANK() OVER (PARTITION BY `fg1`.`date`, `fg1`.`place`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `irisflowers_featurestore`.`streamflow_fg_1` `fg1`
INNER JOIN `irisflowers_featurestore`.`weath

,date,streamflow,place,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,et0_fao_evapotranspiration
0,1671663600000,30.4489,Abisko,-6.4,-11.2,0.6,0.0,0.42,6.0,16.6,36.0,224.0,0.04
1,1671663600000,5.8187,Spånga,5.2,3.6,3.0,1.7,0.00,12.0,18.0,37.8,215.0,0.08
2,1671663600000,5.0213,Uppsala,4.3,3.1,1.0,0.2,0.00,5.0,18.0,32.0,208.0,0.06


---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset you use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- You can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

In [9]:
feature_view.create_training_data()

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5318/jobs/named/streamflow_fv_1_1_create_fv_td_23122022113710/executions


(1, <hsfs.core.job.Job at 0x1b418a784f0>)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 04 </span>

In the next notebook you will train a model on the dataset, that was created in this notebook.